<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor the German Credit Risk model. Use the notebook to enable quality, drift, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. The common configuration JSON, Drift Configuration and Explainability Configuration archives generated by using the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb).
2. Feedback, payload, drifted transactions, explanations queue and result tables in an Apache Hive data warehouse that use the data description language statements (DDLs) that are generated as part of running the previous common configuration notebook.

## Contents

* [1. Setup](#setup)
* [2. Configure Watson OpenScale](#openscale)
* [3. Set up Subscription](#subscription)
* [4. Quality monitoring](#quality)
* [5. Drift monitoring](#drift)
* [6. Fairness monitoring](#fairness)
* [7. Explainability monitoring](#explainability)

# 1. Setup <a name="setup"></a>

## Package installation

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

In [1]:
import warnings
warnings.filterwarnings('ignore')
%env PIP_DISABLE_PIP_VERSION_CHECK=1

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [ ]:
!pip install ibm-watson-openscale --no-cache | tail -n 1

## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [2]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

## Specify model details

### Service provider and subscription metadata

In [3]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [4]:
IAE_SPARK_DISPLAY_NAME = ""
IAE_SPARK_JOBS_ENDPOINT = ""
IBM_CPD_VOLUME = ""
IBM_CPD_USERNAME = ""
IBM_CPD_APIKEY = ""
IAE_SPARK_NAME = ""
IAE_SPARK_DESCRIPTION = ""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [5]:
spark_parameters = {
    "max_num_executors": 2,
    "min_num_executors": 1,
    "executor_cores": 3,
    "executor_memory": 2,
    "driver_cores": 2,
    "driver_memory": 2
}

### Apache Hive

To connect to Apache Hive, you must provide the following details:

- HIVE_CONNECTION_NAME: _Custom display name for the Hive Connection_
- HIVE_CONNECTION_DESCRIPTION: _Custom description for the Hive connection_
- HIVE_METASTORE_URI: _Thrift URI for Hive Metastore to connect to_

In case of a kerberos-enabled hive, provide additional details required to obtain the Hadoop delegation token:
- HIVE_KERBEROS_PRINCIPAL: The kerberos principal used to generate the delegation token
- DELEGATION_TOKEN_SECRET_URN: The secret_urn of the CP4D vault where the token is stored *OR*
- DELEGATION_TOKEN_ENDPOINT: The REST endpoint which generates and returns the delegation token

In [6]:
HIVE_CONNECTION_NAME = ""
HIVE_CONNECTION_DESCRIPTION = ""
HIVE_METASTORE_URI = ""

# Flag to indicate if the Hive is secured with kerberos
KERBEROS_ENABLED = False
# Provide Hadoop delegation token details if KERBEROS_ENABLED is True
# Provide either secret_urn of the CP4D vault OR the delegation token endpoint. One of the two fields is mandatory to fetch the delegation token.
HIVE_KERBEROS_PRINCIPAL = ""
DELEGATION_TOKEN_SECRET_URN = ""
DELEGATION_TOKEN_ENDPOINT = ""

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_DATABASE_NAME: _Database name where feedback table is present_
- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [7]:
#feedback

FEEDBACK_DATABASE_NAME = ""
FEEDBACK_TABLE_NAME = ""

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_DATABASE_NAME: _Database name where payload logging table is present_
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_DATABASE_NAME: _Database name where drifted transactions table is present_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [8]:
#payload logging

PAYLOAD_DATABASE_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_DATABASE_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_DATABASE_NAME: _Database name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [9]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [10]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])
data_mart_id=WOS_CREDENTIALS["instance_id"]

### Display Watson OpenScale datamart details

In [11]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-06-04 05:19:03.698000+00:00,00000000-0000-0000-0000-000000000000


### Create a service provider

In [12]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [13]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        credentials={},
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider a6780fe7-b9aa-4b91-bf87-4ade19fd41ae 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




None,active,WML_IAE,custom_machine_learning,2024-06-23 10:16:49.805000+00:00,a6780fe7-b9aa-4b91-bf87-4ade19fd41ae
99999999-9999-9999-9999-999999999999,active,shreya,watson_machine_learning,2024-06-20 13:22:29.550000+00:00,db6e279f-6d0b-49db-a94f-4b14a12aaddc
00000000-0000-0000-0000-000000000000,active,wml_aie,watson_machine_learning,2024-06-14 06:11:43.041000+00:00,15921037-330f-479a-9b19-2e2285553bf0
00000000-0000-0000-0000-000000000000,active,wml_iae_jdbc,watson_machine_learning,2024-06-14 05:57:05.755000+00:00,33e21c9c-ae94-438a-b322-8827f221b5fe
None,active,SDK_BATCH_SUB,custom_machine_learning,2024-06-14 05:37:53.874000+00:00,210bf9fa-bdb5-4cb8-8bb5-b2f938c1d5dc
None,active,poojitha,custom_machine_learning,2024-06-14 05:34:56.721000+00:00,bdccd48d-6ccb-42bd-8662-dec574aa1b83
00000000-0000-0000-0000-000000000000,active,WML,watson_machine_learning,2024-06-14 04:32:02.681000+00:00,8b392383-39fc-44f5-a1c0-30bdea67f61c
None,active,poojitha2,custom_machine_learning,2024-06-13 15:43:11.394000+00:00,47aeab55-43cc-4271-b86b-cd5b58b3867b
99999999-9999-9999-9999-999999999999,active,notebooks 5.0,watson_machine_learning,2024-06-13 10:28:22.932000+00:00,4d2f2fb2-6b64-4d58-8f13-257166e468e9
None,active,CUSTOM_APIKEY_CLOUD_WITHOUTAPI_PREPROD,custom_machine_learning,2024-06-12 15:52:05.187000+00:00,20890b10-0f37-4ba3-b7e5-1c8ecac028cb


Note: First 10 records were displayed.


In [14]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

{
  "metadata": {
    "id": "a6780fe7-b9aa-4b91-bf87-4ade19fd41ae",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:service_provider:a6780fe7-b9aa-4b91-bf87-4ade19fd41ae",
    "url": "/v2/service_providers/a6780fe7-b9aa-4b91-bf87-4ade19fd41ae",
    "created_at": "2024-06-23T10:16:49.805000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAE",
    "service_type": "custom_machine_learning",
    "credentials": {
      "secret_id": "085d7013-bbeb-4ad5-9fcd-d678ad02cc04"
    },
    "operational_space_id": "production",
    "status": {
      "state": "active"
    }
  }
}


### Create integrated systems for Spark Engine and Hive

In [15]:
# Delete existing spark and hive integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, HIVE_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system Hive_WML_IAE_temp
Deleting integrated system WML_IAEKB_Spark


#### Spark Engine

In [16]:
spark_engine_details = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_engine_id = spark_engine_details.metadata.id
print(spark_engine_details)

{
  "metadata": {
    "id": "c1d21810-40cf-4b24-b291-31fa591479f9",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:c1d21810-40cf-4b24-b291-31fa591479f9",
    "url": "/v2/integrated_systems/c1d21810-40cf-4b24-b291-31fa591479f9",
    "created_at": "2024-07-05T07:15:29.801000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAEKB_Spark",
    "type": "spark",
    "description": "WML_IAEKB_Spark",
    "credentials": {
      "secret_id": "a3b40a6b-6599-4b98-ac40-1020bc3d9949"
    },
    "connection": {
      "display_name": "IAEBatchSpark",
      "endpoint": "https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/v4/analytics_engines/7d2a4875-563e-42d0-994f-518ebf6e1e42/spark_applications",
      "location_type": "cpd_iae",
      "volume": "cpd-instance::IAEBatchTest"
    }
  }
}


#### Hive

In [17]:
hive_connection = {}
hive_credentials = {}
if HIVE_METASTORE_URI is not None:
    hive_connection["metastore_url"] = HIVE_METASTORE_URI
    hive_connection["location_type"] = "metastore"

if KERBEROS_ENABLED is True:
    hive_connection["kerberos_enabled"] = True
    hive_credentials["kerberos_principal"] = HIVE_KERBEROS_PRINCIPAL
    if DELEGATION_TOKEN_SECRET_URN:
        hive_credentials["delegation_token_urn"] = DELEGATION_TOKEN_SECRET_URN
    if DELEGATION_TOKEN_ENDPOINT:
        hive_credentials["delegation_token_endpoint"] = DELEGATION_TOKEN_ENDPOINT

hive_connection_details = IntegratedSystems(wos_client).add(
    name=HIVE_CONNECTION_NAME,
    description=HIVE_CONNECTION_DESCRIPTION,
    type="hive",
    credentials=hive_credentials,
    connection=hive_connection
).result

hive_connection_id=hive_connection_details.metadata.id
print(hive_connection_details)

{
  "metadata": {
    "id": "1bae0ecd-9482-400b-b5f9-58ba4deb9c8e",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:1bae0ecd-9482-400b-b5f9-58ba4deb9c8e",
    "url": "/v2/integrated_systems/1bae0ecd-9482-400b-b5f9-58ba4deb9c8e",
    "created_at": "2024-06-23T10:17:31.494000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "Hive_WML_IAE_temp",
    "type": "hive",
    "description": "Hive_WML_IAE_temp",
    "credentials": {
      "secret_id": "9ac2e403-b293-4736-9bed-e8204ce3cc99"
    },
    "connection": {
      "location_type": "metastore",
      "metastore_url": "thrift://shillong1.fyre.ibm.com:9083"
    }
  }
}


# 3. Set up a subscription<a name="subscription"></a>

In [18]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

4b02f297-e125-4e49-8b33-b0f2db6e8c56,GCR Batch Binary,00000000-0000-0000-0000-000000000000,585ad7c4-3632-43c7-a79a-5ce34931a62c,GCR Batch Binary,db6e279f-6d0b-49db-a94f-4b14a12aaddc,error,2024-06-21 05:03:52.125000+00:00,5afc4862-a4ef-449b-aaf5-c4e0e065c5f6
0cb8cfb5-d6c3-4b00-8f5b-41f28ff4eb98,GCR Batch Subscription-GCR2,00000000-0000-0000-0000-000000000000,10ae0966-6849-449d-a883-48b1e46384a3,GCR Batch Subscription-GCR2,db6e279f-6d0b-49db-a94f-4b14a12aaddc,pending_delete,2024-06-20 14:21:29.556000+00:00,cc0a7062-56ef-4feb-91aa-33764217ad77
6edae2e7-f6c3-465d-8b71-aa0d2455acd7,GCR Batch Subscription-GCR2,00000000-0000-0000-0000-000000000000,7c5c1c58-17d7-4f63-bd6a-0219f07e0fec,GCR Batch Subscription-GCR2,db6e279f-6d0b-49db-a94f-4b14a12aaddc,pending_delete,2024-06-20 14:12:18.807000+00:00,89646855-3162-4b5a-aafe-0a1e8a2a3946
e9a7961c-fa06-4262-9093-aa64b8838c72,GCR Batch Subscription-GCR1,00000000-0000-0000-0000-000000000000,44220a70-386a-4a80-bebc-3348ea0f6088,GCR Batch Subscription-GCR1,db6e279f-6d0b-49db-a94f-4b14a12aaddc,pending_delete,2024-06-20 13:53:56.389000+00:00,d330f8e4-8c21-4613-b957-0fbaea510978
e64eb967-e972-4b10-bc6b-1d5287b5b3b5,GCR Batch Subscription-GCR1,00000000-0000-0000-0000-000000000000,23dbc535-b916-4627-a194-bf8e95cec8ae,GCR Batch Subscription-GCR1,db6e279f-6d0b-49db-a94f-4b14a12aaddc,pending_delete,2024-06-20 13:56:45.938000+00:00,a116845d-0dcc-4371-bf15-a793f6e3eedf
561d39c8-2d53-49b3-9798-3da91452d69c,GCR Batch Subscription-GCR1,00000000-0000-0000-0000-000000000000,76e786b8-af91-469a-9e88-244fbd1b24e9,GCR Batch Subscription-GCR1,db6e279f-6d0b-49db-a94f-4b14a12aaddc,pending_delete,2024-06-20 13:42:16.620000+00:00,ad417850-8aaa-427e-aeba-2e51f1e9fcaf
da2389c8-1670-4e99-880a-758ac53edbd4,GCR Batch Subscription-GCR,00000000-0000-0000-0000-000000000000,c1084e8f-6785-464d-a0a1-c861ad076112,GCR Batch Subscription-GCR,db6e279f-6d0b-49db-a94f-4b14a12aaddc,pending_delete,2024-06-20 13:34:29.100000+00:00,4fd846dc-0098-49e0-a387-b71f7353622c
c5565a45-f66d-4d72-bedf-eb8fcc5d5acd,GCR Batch Subscription-GCR,00000000-0000-0000-0000-000000000000,b17c0ff8-3503-44f8-92c9-f09fa5219381,GCR Batch Subscription-GCR,db6e279f-6d0b-49db-a94f-4b14a12aaddc,pending_delete,2024-06-20 13:24:56.179000+00:00,9acf1d51-2958-44f8-ade9-d6d7663784ef
592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9,gcr - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,2b976af0-e4ab-4859-af7d-2f2287d864ad,gcr model,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-06-14 03:16:35.964000+00:00,7c0c6db0-0c7f-415b-85f1-cae28daded5b
1fd14e8b-ddfd-4bf7-b559-157035c36260,[asset] E2E_ForestCoverageMultiClassBatchIAEDB2,00000000-0000-0000-0000-000000000000,d4f4e04a-2304-43dc-a7c8-bf8c64b6beca,E2E_ForestCoverageMultiClassBatchIAEDB2,502e4610-40a3-41fa-9b5b-5f04e3186159,active,2024-06-17 07:25:57.273000+00:00,b4f91d85-92fb-4629-86b9-ec0d6323905d


Note: First 10 records were displayed.


### Set subscription metadata

In the following cell, type a path to the common configuration JSON file that you created by running the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb). After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [19]:
import uuid

common_configuration = None
with open("/path/to/dir/containing/common_config.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        print("Please provide the correct path to the common configuration JSON")
    
# Set asset details
asset = Asset(
    asset_id=str(uuid.uuid4()),
    url="",
    name=SUBSCRIPTION_NAME,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=str(uuid.uuid4()),
    name=SUBSCRIPTION_NAME,
    description=SUBSCRIPTION_DESCRIPTION,
    deployment_type="batch"
)

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=[common_configuration["probability"]],
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"]
)

# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_engine_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_DATABASE_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=FEEDBACK_DATABASE_NAME,
        schema_name=FEEDBACK_DATABASE_NAME,
        table_name=FEEDBACK_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={
            "hive_storage_format": "" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_DATABASE_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_DATABASE_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=PAYLOAD_DATABASE_NAME,
        schema_name=PAYLOAD_DATABASE_NAME,
        table_name=PAYLOAD_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        parameters={
            "hive_storage_format": "" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift", 
        database_name=DRIFT_DATABASE_NAME,
        schema_name=DRIFT_DATABASE_NAME,
        table_name=DRIFT_TABLE_NAME, 
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_connection_id
        ),
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:

    explainability_queue_data_source = DataSource(
        type="explain_queue", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={
            "hive_storage_format": "" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

In [20]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    data_sources=data_sources).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4",
    "url": "/v2/subscriptions/9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4",
    "created_at": "2024-06-23T10:18:00.421000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "a6780fe7-b9aa-4b91-bf87-4ade19fd41ae",
    "asset": {
      "asset_id": "1995cae7-3390-4dad-a0bc-5d056fe3397f",
      "url": "",
      "name": "WML_IAE",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "label_column": "Risk",
      "prediction_field": "prediction",
      "feature_fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSavings",
 

In [21]:
# Add training, output, and input data schemas to the subscription

training_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/training_data_schema', value=common_configuration["training_data_schema"])
]

input_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/input_data_schema', value=common_configuration["input_data_schema"])
]

output_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/output_data_schema', value=common_configuration["output_data_schema"])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=training_data_schema_patch_document)
wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=input_data_schema_patch_document)
wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=output_data_schema_patch_document)

In [22]:
import time
# Checking subscription status

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(30)

preparing
preparing
active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [24]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 200
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "dd08c0ae-01f9-4f3f-ad7f-471ab4cc5e8f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:dd08c0ae-01f9-4f3f-ad7f-471ab4cc5e8f",
    "url": "/v2/monitor_instances/dd08c0ae-01f9-4f3f-ad7f-471ab4cc5e8f",
    "created_at": "2024-06-23T10:20:49.288000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4"
    },
    "parameters": {
      "min_feedback_data_size": 200
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "re

### Check monitor instance status

In [ ]:
import datetime
quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), quality_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime('%H:%M:%S'), quality_status)

10:21:02 active


In [26]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "dd08c0ae-01f9-4f3f-ad7f-471ab4cc5e8f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:dd08c0ae-01f9-4f3f-ad7f-471ab4cc5e8f",
    "url": "/v2/monitor_instances/dd08c0ae-01f9-4f3f-ad7f-471ab4cc5e8f",
    "created_at": "2024-06-23T10:20:49.288000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-06-23T10:20:49.967000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4"
    },
    "parameters": {
      "min_feedback_data_size": 200
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "

### Run an on-demand evaluation

In [27]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=
                                                          quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "69e4719c-aa63-4e82-ab44-006934a7ca73",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:69e4719c-aa63-4e82-ab44-006934a7ca73",
    "url": "/v2/monitor_instances/30171a44-7a4e-4ebc-afda-3d0de5625cf5/runs/69e4719c-aa63-4e82-ab44-006934a7ca73",
    "created_at": "2024-07-05T08:31:41.408000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "last_processed_time": "2024-07-01T11:20:29.932543Z",
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2024-07-05T08:31:41.396000Z",
      "started_at": "2024-07-05T08:31:41.408000Z",
      "operators": []
    }
  }
}


In [28]:
# Check run status
import time
quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
print(monitoring_run_details.entity)

08:32:16 running
08:32:46 running
08:33:16 running
08:33:46 running
08:34:16 running
08:34:47 running
08:35:17 running
08:35:47 running
08:36:17 running
{
  "parameters": {
    "job_run_schedule_id": "7eb163d5-f842-4b3c-a7b5-a26069a694b0",
    "last_processed_time": "2024-07-01T11:20:29.932543Z",
    "min_feedback_data_size": 10,
    "total_records_processed": 10
  },
  "status": {
    "state": "finished",
    "queued_at": "2024-07-05T08:31:41.396000Z",
    "started_at": "2024-07-05T08:31:41.972000Z",
    "updated_at": "2024-07-05T08:36:43.802000Z",
    "completed_at": "2024-07-05T08:36:43.752000Z",
    "message": "Quality execution is completed.",
    "operators": [
      {
        "id": "original",
        "status": {
          "state": "finished",
          "completed_at": "2024-07-05T08:36:43.752000Z"
        }
      }
    ]
  }
}
08:36:47 finished


### Display quality metrics

In [29]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-07-05 08:36:43.533411+00:00,true_positive_rate,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.3333333333333333,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,area_under_roc,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.5952380952380952,0.8,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,precision,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.5,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,f1_measure,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.4,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,accuracy,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.7,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,log_loss,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.715401570634091,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,false_positive_rate,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.14285714285714285,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,area_under_pr,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.43333333333333335,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-05 08:36:43.533411+00:00,recall,b2f58f89-2d32-4a7f-bdcc-92870cf4317c,0.3333333333333333,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,69e4719c-aa63-4e82-ab44-006934a7ca73,subscription,3046b546-e422-4737-9123-a26e025d3cef
2024-07-01 11:20:29.554742+00:00,true_positive_rate,88104bb6-8c17-45ed-994f-e2583ec218b2,0.3333333333333333,None,None,['model_type:original'],quality,30171a44-7a4e-4ebc-afda-3d0de5625cf5,40bf945a-b5fc-44fd-9aa7-a152d17d1d4f,subscription,3046b546-e422-4737-9123-a26e025d3cef


Note: First 10 records were displayed.


# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [30]:
wos_client.monitor_instances.upload_drift_model(
    model_path="/path/to/dir/containing/drift.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{'data_constraints': {'id': '9dd70e27-eab3-4da5-8e46-4b5f8954fea2',
  'version': '0.02_batch',
  'columns': [{'name': 'Age',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CheckingStatus',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CreditHistory',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CurrentResidenceDuration',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'Dependents',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'EmploymentDuration',
    'dtype': 'categorical',
    'count': 1000,
    'appr

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of records that triggers the drift monitor to run. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to re-train the model based on the drift analysis.

In [31]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "b4b2be03-29aa-4483-bea1-2778d98809d2",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:b4b2be03-29aa-4483-bea1-2778d98809d2",
    "url": "/v2/monitor_instances/b4b2be03-29aa-4483-bea1-2778d98809d2",
    "created_at": "2024-06-23T11:36:27.358000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 100,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_d

### Check monitor instance status

In [32]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)

06:48:42 active


### Run an on-demand evaluation

In [33]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "b4b2be03-29aa-4483-bea1-2778d98809d2",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:b4b2be03-29aa-4483-bea1-2778d98809d2",
    "url": "/v2/monitor_instances/b4b2be03-29aa-4483-bea1-2778d98809d2",
    "created_at": "2024-06-23T11:36:27.358000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-06-24T09:47:55.195000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0

In [34]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "c5371dee-c69a-481c-9ffa-3ba36b454c38",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:c5371dee-c69a-481c-9ffa-3ba36b454c38",
    "url": "/v2/monitor_instances/b4b2be03-29aa-4483-bea1-2778d98809d2/runs/c5371dee-c69a-481c-9ffa-3ba36b454c38",
    "created_at": "2024-06-25T07:16:16.719000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0.05,
      "last_drift_run_end_timestamp": "2024-06-25T07:11:54.209506Z",
      "last_drift_run_start_timestamp": "2024-06-25T06:52:58.500092Z",
      "last_record_window_end_timestamp": "2024-06-25T06:52:59.834863Z",
      "last_record_window_start_timestamp": "2024-06-25T06

In [35]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)


07:16:19 running
07:16:49 running
07:17:19 running
07:17:50 running
07:18:20 running
07:18:50 finished


### Display drift metrics

In [36]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-06-25 07:16:18.438142+00:00,data_drift_magnitude,7b671463-47e5-4d92-bf6a-ad843fe11f5d,0.0,None,0.1,[],drift,b4b2be03-29aa-4483-bea1-2778d98809d2,c5371dee-c69a-481c-9ffa-3ba36b454c38,subscription,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4
2024-06-25 07:16:18.438142+00:00,drift_magnitude,7b671463-47e5-4d92-bf6a-ad843fe11f5d,0.0,None,0.05,[],drift,b4b2be03-29aa-4483-bea1-2778d98809d2,c5371dee-c69a-481c-9ffa-3ba36b454c38,subscription,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4
2024-06-25 07:16:18.438142+00:00,predicted_accuracy,7b671463-47e5-4d92-bf6a-ad843fe11f5d,0.8181818181818182,None,None,[],drift,b4b2be03-29aa-4483-bea1-2778d98809d2,c5371dee-c69a-481c-9ffa-3ba36b454c38,subscription,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4


# 6. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [37]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = configuration_json["fairness_configuration"]["parameters"]
thresholds = configuration_json["fairness_configuration"]["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "8222b267-c556-48b7-b940-fb5c9be3c846",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:8222b267-c556-48b7-b940-fb5c9be3c846",
    "url": "/v2/monitor_instances/8222b267-c556-48b7-b940-fb5c9be3c846",
    "created_at": "2024-06-25T07:30:45.674000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4"
    },
    "parameters": {
      "training_data_last_processed_time": "2024-06-23T08:55:38.198019Z",
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "majority": [
            "male"
          ],
          "t

### Check monitor instance status

In [38]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

07:30:51 preparing


07:31:21 active


### Run an on-demand evaluation

In [39]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "716739e9-1bbe-4cf1-b61d-c9d960cd91e8",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:716739e9-1bbe-4cf1-b61d-c9d960cd91e8",
    "url": "/v2/monitor_instances/8222b267-c556-48b7-b940-fb5c9be3c846/runs/716739e9-1bbe-4cf1-b61d-c9d960cd91e8",
    "created_at": "2024-06-25T07:32:37.436000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "minority": [
            "female"
          ],
          "threshold": 0.95
        },
        {
          "feature": "Age",
          "majority": [
            [
              26,
              75
            ]
          ],
          "metric_ids": [
            "stati

In [40]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

07:32:45 running
07:33:15 running
07:33:45 running
07:34:15 running
07:34:45 finished


### Display fairness metrics

In [41]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-06-25 07:37:40.538314+00:00,fairness_value,0cbe34c7-5532-428b-b537-6179e91d0f7b,175.00000000000003,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,8222b267-c556-48b7-b940-fb5c9be3c846,716739e9-1bbe-4cf1-b61d-c9d960cd91e8,subscription,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4
2024-06-25 07:37:40.538314+00:00,statistical_parity_difference,0cbe34c7-5532-428b-b537-6179e91d0f7b,0.429,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,8222b267-c556-48b7-b940-fb5c9be3c846,716739e9-1bbe-4cf1-b61d-c9d960cd91e8,subscription,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4
2024-06-25 07:37:40.538314+00:00,fairness_value,0cbe34c7-5532-428b-b537-6179e91d0f7b,150.00000000000003,95.0,125.0,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,8222b267-c556-48b7-b940-fb5c9be3c846,716739e9-1bbe-4cf1-b61d-c9d960cd91e8,subscription,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4
2024-06-25 07:37:40.538314+00:00,statistical_parity_difference,0cbe34c7-5532-428b-b537-6179e91d0f7b,0.333,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,8222b267-c556-48b7-b940-fb5c9be3c846,716739e9-1bbe-4cf1-b61d-c9d960cd91e8,subscription,9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4


# 7. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [42]:
with open("/path/to/dir/containing/explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability archive successfully.")

Uploaded explainability archive successfully.


In [43]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
# Uncomment the below lines to enable lime global explanation. Available from Cloud Pak for Data version 4.6.4 onwards.
#    "global_explanation": {
#        "enabled": True,  # Flag to enable global explanation 
#        "explanation_method": "lime",
#        "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
#    }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "061d91a4-3325-40e4-9a16-fbec51971eb9",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:061d91a4-3325-40e4-9a16-fbec51971eb9",
    "url": "/v2/monitor_instances/061d91a4-3325-40e4-9a16-fbec51971eb9",
    "created_at": "2024-06-25T07:43:05.412000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4"
    },
    "parameters": {},
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "repeat_type": "week"
 

### Check monitor instance status

In [44]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

08:22:24 active


### Run an on-demand evaluation

In [45]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "061d91a4-3325-40e4-9a16-fbec51971eb9",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:061d91a4-3325-40e4-9a16-fbec51971eb9",
    "url": "/v2/monitor_instances/061d91a4-3325-40e4-9a16-fbec51971eb9",
    "created_at": "2024-06-25T07:43:05.412000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-06-25T08:21:20.917000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4"
    },
    "parameters": {
      "config_modified_at": "2024-06-25T08:21:20.860964Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "perturbations_count": 20

In [46]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "8a3bfc1e-2266-4dc2-b3ac-66db87b59c0a",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:8a3bfc1e-2266-4dc2-b3ac-66db87b59c0a",
    "url": "/v2/monitor_instances/061d91a4-3325-40e4-9a16-fbec51971eb9/runs/8a3bfc1e-2266-4dc2-b3ac-66db87b59c0a",
    "created_at": "2024-06-25T08:44:13.729000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_modified_at": "2024-06-25T08:43:48.958184Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "perturbations_count": 208
      },
      "validate_table_job_app_id": null,
      "validate_table_job_id": "9eb53b05-abd8-433e-921c-c2f5cc988bb1",
      "validate_table_job_output_path": "explainability_configuration/02f38fbf-f155-4e5b-9de7-79e56d0d868a/output/2c1ab646-7407-479e-8946-82c39825f31d",
      "va

In [47]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
    elif explainability_run_status == "error":
        print(monitoring_run_details.entity)
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

08:44:15 running
08:45:15 running
08:46:15 running
08:47:15 running
08:48:15 running
08:49:16 running
08:50:16 finished


In [ ]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
# wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

### Display sample explanations

In [48]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result
print(explanations)

{
  "total_count": 33,
  "limit": 50,
  "offset": 0,
  "explanation_fields": [
    "explanation_task_id",
    "scoring_id",
    "created_at",
    "finished_at",
    "status",
    "prediction",
    "subscription_id",
    "deployment_id",
    "asset_name",
    "deployment_name",
    "probability",
    "explanation_type"
  ],
  "explanation_values": [
    [
      "e4f3e345-1d92-49c3-b4e8-30e85a398090",
      "sc2",
      "2024-06-25T08:45:30.831194Z",
      "2024-06-25T08:45:30.831400Z",
      "finished",
      "No Risk",
      "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4",
      "88e7e864-328f-4b3f-a683-5de9849d93e2",
      "WML_IAE",
      "WML_IAE",
      0.8291593,
      "lime"
    ],
    [
      "f36a077d-d616-404a-8c5f-e9cd70ef8025",
      "sc3",
      "2024-06-25T08:45:30.839126Z",
      "2024-06-25T08:45:30.839573Z",
      "finished",
      "No Risk",
      "9bb07cbc-82ca-4bc6-a22a-4bb36fcdd5e4",
      "88e7e864-328f-4b3f-a683-5de9849d93e2",
      "WML_IAE",
      "WML_IAE",
      0.7597

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the **German Credit model** to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.